# Data Analysis - Oldies

## Preparación

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import altair as alt

# Configuración Jupyter
pd.set_option('display.max_columns', None)
sns.set_style("darkgrid")

# Cargamos el dataframe ya limpio desde el análisis de Features
data_ana_corona = pd.read_csv("data_ana_coronaperiod_withgenre.csv", sep = ',', parse_dates=['release_date_year'])
# data_ana_corona['release_date_year'] = data_ana_corona['release_date_year'].dt.year
data_ana_corona = data_ana_corona.drop(columns=['Unnamed: 0'])

# Cargamos el dataframe ya limpio desde el análisis de Features
data_ana_normal = pd.read_csv("data_ana_normalperiod.csv", sep = ',', parse_dates=['release_date_year'])

print('Tamaño inicial: ',data_ana_corona.shape)
data_ana_corona.sample(2)

Tamaño inicial:  (1581, 39)


,Track Name,Artist,Streams,date,spotify_id,year,Streamstotal,album,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,time_signature,release_date_year,genre1,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10,genres_together,artist_id,artist_followers,artist_popularity,streamstotal_weights,genre_depurated,genre_depurated_final
1013,Échame La Culpa,Luis Fonsi,172911,2018-03-13,1zsG4eaZmkA1dvjDDsAGLK,2018,9564671,Échame La Culpa,2017-11-17,0.255120,0.06,0.0322,0.724,0.895,0.0,0.097,0.900361,0.025912,0.640,0.224332,4,2017-01-01,latin,latin pop,puerto rican pop,tropical,NaN,NaN,NaN,NaN,NaN,NaN,"latin,latin pop,puerto rican pop,tropical",4V8Sr092TqfHkfAA5fXXqG,8731720,0.808081,0.013590,puerto rican pop,pop
360,Bailando,Nil Moliner,31397,2020-03-15,27NTdNyp9n5B4lswgGikG3,2020,1869635,Bailando en la batalla,2020-02-21,0.357554,0.57,0.1320,0.718,0.742,0.0,0.298,0.829974,0.034019,0.944,0.518531,4,2020-01-01,latin viral pop,musica catalana,spanish pop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"latin viral pop,musica catalana,spanish pop",6X8uzem5Dbl4l2so4CeexD,102383,0.676768,0.001669,latin viral pop,pop


# Pregunta 3: ¿Se han vuelto a reescuchar canciones antiguas en el confinamiento?, ¿más que en otros años?

La idea de esta pregunta es saber si durante el periodo del coronavirus hemos rememorado canciones antiguas, hasta el punto de llegar al top200 y saber si es algo puntual u ocurre en otros años también. Para ello vamos a usar la columna "release_date_year" que nos dice la fecha de lanzamiento de la canción.

Primero vamos a hacer un conteo de canciones de cada año.

In [3]:
# Agrupación de fechas
fecha_1 = datetime(1959,1,1)
fecha_2 = datetime(2014,1,1)
fecha_3 = datetime(2019,1,1)

# Vamos a crear un dataframe agrupando los datos que nos interesan y vemos cuantas canciones hay de cada Release date en cada año
data_oldies_count = data_ana_corona.groupby(['year', 'release_date_year']).count()
data_oldies_count.reset_index(inplace=True)
data_oldies_count = data_oldies_count[['year', 'release_date_year', 'Track Name']]
data_oldies_count.rename(columns={'Track Name': 'Total_songs'}, inplace=True)

# Gráfico
data_oldies_count['year'] = data_oldies_count['year'].astype(str)
alt.Chart(data_oldies_count, title='Número de canciones hasta 2019').mark_circle(
size=100,
).encode(
    x = alt.X('year(release_date_year):T', title='year', bin=alt.Bin(maxbins=15)),
    y= alt.Y('Total_songs', title='Número de canciones'),
    color=alt.Color('year', title='Año de los datos'),
).properties(
    width=800,
    height=150).interactive(
)


alt.Chart(...)

Como está un tanto disperso, he decidido agrupar los años en dos rangos de fechas de las canciones antiguas: hasta 2014 y de 2014 a 2019. Ojo! también hay que tener en cuenta que los años más actuales tendrán más canciones antiguas porque cogen las de los años anteriores.

En un primer vistazo ya vemos que en el 2020 se han reescuchado más canciones antiguas que otros años.

In [4]:
# Ploteamos el rango más antiguo para ver donde están las canciones por año
data_oldies_count_1959_2014 = data_oldies_count[data_oldies_count.release_date_year.between(fecha_1, fecha_2)]
graph_oldies_count_1959_2014 = alt.Chart(data_oldies_count_1959_2014, title='Número de canciones hasta 2014').mark_bar().encode(
    x = 'release_date_year',
    y= alt.Y('Total_songs', title='Número de canciones'),
    color=alt.Color('year', title='Año de los datos'),
).properties(width=800, height=300).interactive()

data_oldies_count_2014_2019 = data_oldies_count[data_oldies_count.release_date_year.between(fecha_2, fecha_3)]
graph_oldies_count_2014_2019 = alt.Chart(data_oldies_count_2014_2019, title='Número de canciones entre 2014-2019').mark_bar().encode(
    x = 'release_date_year',
    y= alt.Y('Total_songs', title='Número de canciones'),
    color=alt.Color('year', title='Año de los datos'),
).properties(width=800, height=300).interactive()

graph_oldies_count_1959_2014&graph_oldies_count_2014_2019

alt.VConcatChart(...)

Pero ojo, estamos mirando número de canciones y no Streamstotales de las mismas, que es lo que de verdad nos interesa. Entonces, hacemos lo mismo con Streams totales y vemos que el gráfico coge otro matiz, determinadas canciones antiguas se hacen más visibles.A simple vista parece que en 2019 (en periodo pandemia) se escucharon más canciones de lso 70-80 y en 2020 del 2000 en adelante. Para saber qué canción corresponde a cada datos, posar el ratón sobre el gráfico.

In [5]:
data_oldies_streams = data_ana_corona.groupby(['year', 'release_date_year', 'Track Name']).sum()
data_oldies_streams.reset_index(inplace=True)
data_oldies_streams = data_oldies_streams[['year', 'release_date_year', 'Track Name','Streamstotal']]
data_oldies_streams['year'] = data_oldies_streams['year'].astype(str)

# Ploteamos el rango más antiguo para ver donde están las canciones por año
data_oldies_streams_1959_2014 = data_oldies_streams[data_oldies_streams.release_date_year.between(fecha_1, fecha_2)]
graph_oldies_streams_1959_2014 = alt.Chart(data_oldies_streams_1959_2014, title='Streams totales canciones hasta 2014').mark_bar().encode(
    x = 'release_date_year',
    y= alt.Y('Streamstotal', title='Streams totales'),
    color=alt.Color('year', title='Año de los datos'),
    tooltip= 'Track Name',
).properties(width=800, height=300).interactive()

data_oldies_streams_2014_2019 = data_oldies_streams[data_oldies_streams.release_date_year.between(fecha_2, fecha_3)]
graph_oldies_streams_2014_2019 = alt.Chart(data_oldies_streams_2014_2019, title='Streams totales canciones 2014-2019').mark_bar().encode(
    x = 'release_date_year',
    y= alt.Y('Streamstotal', title='Streams totales'),
    color=alt.Color('year', title='Año de los datos'),
    size= 'Streamstotal',
    tooltip= 'Track Name',
).properties(width=800, height=300).interactive()

graph_oldies_streams_1959_2014&graph_oldies_streams_2014_2019

alt.VConcatChart(...)

Hacemos un gráfico para ver canciones lanzadas en el rango 1 (hasta 2014) y se ve claramente que en el 2020 ha habido muchas más reproducciones de canciones anteriores al 2014 que los años anteriores. **Aquí si podemos establecer una conclusión más clara: en la pandemia escuchamos más a los oldies, ¿por qué?**

Por otro lado, porque no me quiero quedar con dudas con el gráfico anterior, voy a hacer esta misma comparación con el PERIODO NORMAL que ya tenemos preparado anteriormente. Aplicamos la función y vemos la diferencia, y confirmamos que la diferencia del 2019 al 2020 es mucho mayor durante el periodo de pandemia. ¿Puede estar relacionado con que este periodo de normalidad coge el rango de las navidades donde se hacen muchos karaokes donde evidentemente se escuchan éxitos antiguos? Pero tengo mis dudas porque el periodo normal coge del 8 de Enero al 8 de Marzo. Aún así los streams totales siguen siendo más altos en el periodo pandemia que en el periodo normal en el 2020.

In [28]:
#Creamos una función que saca el dataframe

def oldies_extract(period_dataframe):
    # Creamos fechas para agrupar
    fecha_1 = datetime(1959, 1, 1)
    fecha_2 = datetime(2014, 1, 1)

    # Cogemos datos que nos interesan
    data_oldies = period_dataframe.groupby(['year', 'release_date_year']).sum()
    data_oldies.reset_index(inplace=True)
    data_oldies = data_oldies[['year', 'release_date_year', 'Streamstotal', 'streamstotal_weights']]

    data_oldies_2017 = data_oldies[data_oldies['year'] == 2017]
    data_oldies_2018 = data_oldies[data_oldies['year'] == 2018]
    data_oldies_2019 = data_oldies[data_oldies['year'] == 2019]
    data_oldies_2020 = data_oldies[data_oldies['year'] == 2020]

    # Hacemos la suma de Streams totales
    data_oldies_2017_rango1_totalStreams = data_oldies_2017[data_oldies_2017.release_date_year.between(fecha_1, fecha_2)].sum()['Streamstotal']
    data_oldies_2018_rango1_totalStreams = data_oldies_2018[data_oldies_2018.release_date_year.between(fecha_1, fecha_2)].sum()['Streamstotal']
    data_oldies_2019_rango1_totalStreams = data_oldies_2019[data_oldies_2019.release_date_year.between(fecha_1, fecha_2)].sum()['Streamstotal']
    data_oldies_2020_rango1_totalStreams = data_oldies_2020[data_oldies_2020.release_date_year.between(fecha_1, fecha_2)].sum()['Streamstotal']

    # Create dataframe
    columns = ['Streamstotal']
    index = ['2017', '2018', '2019', '2020']
    data = [data_oldies_2017_rango1_totalStreams,
            data_oldies_2018_rango1_totalStreams,
            data_oldies_2019_rango1_totalStreams,
            data_oldies_2020_rango1_totalStreams]
    data_oldies_rango1 = pd.DataFrame(data, index=index, columns=columns)
    return data_oldies_rango1


data_oldies_rango1_pandemia = oldies_extract(data_ana_corona)
data_oldies_rango1_normal = oldies_extract(data_ana_normal)


#Gráfico
graph_oldies_rango1_pandemia = alt.Chart(data_oldies_rango1_pandemia.reset_index(),
                                         title="Periodo PANDEMIA (en rojo) + Periodo NORMAL (en azul y menos ancha) de música antigua hasta 2014").mark_bar(
color = 'red',
opacity=0.7
).encode(
    x = alt.X('index', title='Años de los datos'),
    y = alt.X('Streamstotal', title='Streams totales'),
).properties(width=700, height=400).interactive()

graph_oldies_rango1_normal = alt.Chart(data_oldies_rango1_normal.reset_index()).mark_bar(
color = 'blue',
opacity=0.5,
size=90
).encode(
    x = 'index',
    y = 'Streamstotal',
).properties(width=700, height=400).interactive()

graph_oldies_rango1_normal+graph_oldies_rango1_pandemia

#Exportar a CSV PARA DATAWRAPPER
'''graph_datawrapper = pd.merge(data_oldies_rango1_pandemia, data_oldies_rango1_normal, left_index=True,
                right_index=True).rename(columns={"Streamstotal_x": "Streams Totales Periodo pandemia", "Streamstotal_y": "Streams Totales Periodo normal"})

graph_datawrapper.to_csv("graphs_article/graph_oldies_songs_2020_streamstotal.csv", sep = ',')
'''

In [20]:
oldies_extract(data_ana_corona)

,Streamstotal
2017,1907248.0
2018,5563131.0
2019,4870357.0
2020,17990795.0


Si volvemos al análisis del periodo pandemia y hacemos lo mismo con el rango 2 (canciones posteriores al 2014), vemos otro panorama, con una tendencia pero claramente está influenciada por el hecho de que años más actuales congregan los años anteriores próximos.

In [7]:
# Para agrupar los años antiguos y sumar sus StreamsTotales

# Creamos fechas para agrupar
fecha_2 = datetime(2014, 1, 1)
fecha_3 = datetime(2020, 1, 1)

data_oldies = data_ana_corona.groupby(['year', 'release_date_year']).sum()
data_oldies.reset_index(inplace=True)
data_oldies = data_oldies[['year', 'release_date_year', 'Streamstotal', 'streamstotal_weights']]

data_oldies_2017 = data_oldies[data_oldies['year'] == 2017]
data_oldies_2018 = data_oldies[data_oldies['year'] == 2018]
data_oldies_2019 = data_oldies[data_oldies['year'] == 2019]
data_oldies_2020 = data_oldies[data_oldies['year'] == 2020]

data_oldies_2017_rango1_totalStreams = data_oldies_2017[data_oldies_2017.release_date_year.between(fecha_2, fecha_3)].sum()['Streamstotal']
data_oldies_2018_rango1_totalStreams = data_oldies_2018[data_oldies_2018.release_date_year.between(fecha_2, fecha_3)].sum()['Streamstotal']
data_oldies_2019_rango1_totalStreams = data_oldies_2019[data_oldies_2019.release_date_year.between(fecha_2, fecha_3)].sum()['Streamstotal']
data_oldies_2020_rango1_totalStreams = data_oldies_2020[data_oldies_2020.release_date_year.between(fecha_2, fecha_3)].sum()['Streamstotal']

# Create dataframe
columns = ['Streamstotal']
index = ['2017', '2018', '2019', '2020']
data = [data_oldies_2017_rango1_totalStreams,
        data_oldies_2018_rango1_totalStreams,
        data_oldies_2019_rango1_totalStreams,
        data_oldies_2020_rango1_totalStreams]
data_oldies_rango2 = pd.DataFrame(data, index=index, columns=columns)

'''# Hacemos un merge con el anterior
data_oldies_rango1andrango2 = pd.merge(data_oldies_rango1, data_oldies_rango2, left_index=True, right_index=True)
data_oldies_rango1andrango2.columns = ['Streams Totales Periodo Pandemia', 'Streams Totales Periodo Normal']'''

#Gráfico
graph_data_oldies_rango2 = alt.Chart(data_oldies_rango2.reset_index()).mark_bar(
color='blue'
).encode(
    x = 'index',
    y = 'Streamstotal',
    color= 'index'
).properties(width=500, height=200).interactive()
graph_data_oldies_rango2


alt.Chart(...)

## Extra: Curiosidad

Para matar nuestra curiosidad, vemos cuales son las canciones antiguas más escuchadas en cada año. En los gráficos, el Artista está en color y en la etiqueta el nombre de la canción.

En 2017, de todo un poco, pero el primero está EdSheran que sacó nuevo disco el 3 de marzo de 2017, lo que es lógico que se vuelva a escuchar alguna canción que otra antigua suya.

In [8]:
#Creo función para sacar por años
def extract_oldies_songs(year):
    data_oldies_songs = data_ana_corona[data_ana_corona.release_date_year.between(fecha_1, fecha_2)]
    data_oldies_songs = data_oldies_songs[data_oldies_songs['year'] == year]
    data_oldies_songs.sort_values(by=['Streamstotal'], ascending=False)
    return data_oldies_songs

In [9]:
# Ploteamos el rango más antiguo para ver donde están las canciones por año
graph_oldies_songs_2017 = alt.Chart(extract_oldies_songs(2017), title="Canciones antiguas más escuchadas en 2017 (periodo pandemia)").mark_circle(
size=100,
).encode(
    x = 'Streamstotal',
    y= 'release_date_year',
    color= 'Artist',
    tooltip= 'Track Name',
).properties(width=700, height=200).interactive()

graph_oldies_songs_2017


alt.Chart(...)

En 2018, hay muchas canciones de Avicii, que tiene lógica porque se suicidó el 20 de abril de 2018

In [10]:
# Gráfico
graph_oldies_songs_2018 = alt.Chart(extract_oldies_songs(2018), title="Canciones antiguas más escuchadas en 2018 (periodo pandemia)").mark_circle(
size=100,
).encode(
    x = 'Streamstotal',
    y= 'release_date_year',
    color= 'Artist',
    tooltip= 'Track Name',
).properties(width=700, height=200).interactive()

graph_oldies_songs_2018

alt.Chart(...)

En 2019 vemos que Queen lo petó, muy lógicamente porque el 31 de octubre se estrenó la película Bohemian Rhapsody que tuvo mucho éxito

In [11]:
# Gráfico
graph_oldies_songs_2019 = alt.Chart(extract_oldies_songs(2019), title="Canciones antiguas más escuchadas en 2019 (periodo pandemia)").mark_circle(
size=100,
).encode(
    x = 'Streamstotal',
    y= 'release_date_year',
    color= 'Artist',
    tooltip= 'Track Name',
).properties(width=700, height=200).interactive()

graph_oldies_songs_2019


alt.Chart(...)

Por último en 2020 vemos que es donde más variedad de reescucha y además de muchos clásicos y en español, entre ellos, claro está, el Resistiré de Dúo Dinámico y que ya analicé y  <a href="https://www.akakicreations.com/es-resistire-una-de-las-canciones-mas-escuchadas-en-espana-durante-la-cuarentena/" target="_blank">publiqué en mi blog</a> hace unos meses y que me inspiró para hacer este trabajo.

**¿Quiere decir esto que en momentos de crisis escuchamos canciones que nos lleven a rememorar buenos momentos? ¿o canciones que nos gusta cantar?, ¿o canciones que ya conocemos de sobra? Hay que acordarse de que durante el confinamiento, a las 20h de cada día la gente salía a los balcones a aplaudir y a veces ponía musica y generalmente era música que se podía cantar y conocida** Por supuesto el 20 abril de Celtas Cortos está presente, pero ojo! porque en años anteriores no lo estuvo. Claramente el confinamiento impulsó la esucha de esta canción y sí, me arriesgo a decir que la melancolía que nos dice la canción del 20 de abril, nos llegó dentro en el tiempo de encierro.

In [13]:
# Gráfico
graph_oldies_songs_2020 = alt.Chart(extract_oldies_songs(2020), title="Canciones antiguas más escuchadas en 2020 (periodo pandemia)").mark_circle(
size=100,
).encode(
    x = 'Streamstotal',
    y= 'release_date_year',
    color= 'Track Name',
    tooltip= 'Track Name',
).properties(width=700, height=200).interactive()

#Exportar a CSV PARA DATAWRAPPER
'''extract_oldies_songs(2020).to_csv("graphs_article/graph_oldies_songs_2020.csv", sep = ',')
'''
graph_oldies_songs_2020


alt.Chart(...)

In [15]:
# Gráfico
graph_oldies_songs_2019 = alt.Chart(extract_oldies_songs(2019), title="Canciones antiguas más escuchadas en 2019 (periodo pandemia)").mark_circle(
size=100,
).encode(
    x = 'Streamstotal',
    y= 'release_date_year',
    color= 'Track Name',
    tooltip= 'Track Name',
).properties(width=700, height=200).interactive()

graph_oldies_songs_2019

alt.Chart(...)

In [16]:
# Gráfico
graph_oldies_songs_2018 = alt.Chart(extract_oldies_songs(2018), title="Canciones antiguas más escuchadas en 2018 (periodo pandemia)").mark_circle(
size=100,
).encode(
    x = 'Streamstotal',
    y= 'release_date_year',
    color= 'Track Name',
    tooltip= 'Track Name',
).properties(width=700, height=200).interactive()

graph_oldies_songs_2018

alt.Chart(...)

Para terminar, ¿cuáles son los éxitos clásicos reescuchados en periodo normalizado? Sorpresa! 3 de las canciones antiguas más escuchadas se mantienen, es decir, antes de la pandemia ya se escuchó mucho Princesas y Soldadito Marinero. Interesante ante todo. Entonces no queda claro hasta que punto un estado de crisis influye en la escucha de los exitosos clásicos o simplemente que ciertas canciones lo vuelven a petar en el año por otras circunstancias.

In [17]:
data_oldies_songs_normal = data_ana_normal[data_ana_normal.release_date_year.between(fecha_1, fecha_2)]
data_oldies_songs_normal_2020= data_oldies_songs_normal[data_oldies_songs_normal['year'] == 2020]
data_oldies_songs_normal_2020.sort_values(by=['Streamstotal'], ascending=False)

# Gráfico
graph_oldies_songs_normal_2020 = alt.Chart(data_oldies_songs_normal_2017, title="Canciones antiguas más escuchadas en 2020 (periodo normal)").mark_circle(
size=100,
).encode(
    x = 'Streamstotal',
    y= 'release_date_year',
    color= 'Track Name',
    tooltip= 'Track Name',
).properties(width=700, height=200).interactive()

graph_oldies_songs_normal_2020

alt.Chart(...)

Por último, vamos a hacer un análisis por generos. Vemos que las canciones que escuchamos más antiguas suelen situarse en cantautor, mientras que las no tan antiguas hay una variedad más amplia.

In [39]:
data_oldies_streams_1959_2014 = data_ana_corona[data_ana_corona.release_date_year.between(fecha_1, fecha_2)]
data_oldies_streams_2014_2019 = data_ana_corona[data_ana_corona.release_date_year.between(fecha_2, fecha_3)]

# Análisis por genre
graph_oldies_genre_1959_2014 = alt.Chart(data_oldies_streams_1959_2014, title='Streams totales canciones hasta 2014').mark_bar(
).encode(
    x = 'genre_depurated_final',
    y= 'Streamstotal',
    color= 'genre_depurated_final',
    tooltip= 'year',
).properties(width=300, height=500).interactive()

graph_oldies_genre_2014_2019 = alt.Chart(data_oldies_streams_2014_2019, title='Streams totales canciones 2014-2019').mark_bar(
).encode(
    x = 'genre_depurated_final',
    y= 'Streamstotal',
    color= 'genre_depurated_final',
    tooltip= 'year',
).properties(width=300, height=500).interactive()

graph_oldies_genre_1959_2014|graph_oldies_genre_2014_2019

alt.HConcatChart(...)

## Conclusión: 

Como ya se ha visto anteriormente: Sí, en el confinamiento se escuchó más música antigua (anterior a 2014) que en años anteriores que es lo que refleja claramente el gráfico siguiente. Y que esa música está más ubicada en el género cantautor.

In [40]:
graph_oldies_rango1_normal+graph_oldies_rango1_pandemia

alt.LayerChart(...)

## Después de este análisis pasamos a los siguientes:
- [Data_Analysis4Extra_Playlists.ipynb](Data_Analysis4Extra_Playlists.ipynb)